### Thesis notebook 4.3. - NOVA IMS

#### LSTM - Temporal data representation

In this notebook, we will finally start our application of temporal representation using LSTMs.
The argument for the usage of Deep Learning stems from the fact that sequences themselves encode information that can be extracted using Recurrent Neural Networks and, more specifically, Long Short Term Memory Units.

#### First Step: Setup a PyTorch environment that enables the use of GPU for training. 

The following cell wll confirm that the GPU will be the default device to use.

In [1]:
import torch
import pycuda.driver as cuda

cuda.init()
## Get Id of default device
torch.cuda.current_device()
# 0
cuda.Device(0).name() # '0' is the id of your GPU

#set all tensors to gpu
torch.set_default_tensor_type('torch.cuda.FloatTensor')

#### Second Step: Import the relevant packages and declare global variables

In [2]:
#import necessary modules/libraries
import numpy as np
import scipy as sc
import pandas as pd
import datetime as dt
import warnings
import time


#tqdm to monitor progress
from tqdm.notebook import tqdm, trange
tqdm.pandas(desc="Progress")

#time related features
from datetime import timedelta
from copy import copy, deepcopy

#vizualization
import matplotlib.pyplot as plt
import seaborn as sns

#imblearn, scalers, kfold and metrics 
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer,PowerTransformer
from sklearn.model_selection import train_test_split, RepeatedKFold, StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, recall_score, classification_report, average_precision_score, precision_recall_curve
from sklearn.pipeline import Pipeline

#import torch related
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable 
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from skorch import NeuralNetClassifier

#and optimizer of learning rate
from torch.optim.lr_scheduler import ReduceLROnPlateau

#import pytorch modules
warnings.filterwarnings('ignore')

In [3]:
#global variables that may come in handy


#colors for vizualizations
nova_ims_colors = ['#BFD72F', '#5C666C']

#standard color for student aggregates
student_color = '#474838'

#standard color for course aggragates
course_color = '#1B3D2F'

#standard continuous colormap
standard_cmap = 'viridis_r'

#Function designed to deal with multiindex and flatten it
def flattenHierarchicalCol(col,sep = '_'):
    '''converts multiindex columns into single index columns while retaining the hierarchical components'''
    if not type(col) is tuple:
        return col
    else:
        new_col = ''
        for leveli,level in enumerate(col):
            if not level == '':
                if not leveli == 0:
                    new_col += sep
                new_col += level
        return new_col
    
#number of replicas - number of repeats of stratified k fold - in this case 10
replicas = 30

#names to display on result figures
date_names = {
             'Date_threshold_10': '10% of Course Duration',   
             'Date_threshold_25': '25% of Course Duration', 
             'Date_threshold_33': '33% of Course Duration', 
             'Date_threshold_50': '50% of Course Duration', 
             'Date_threshold_100':'100% of Course Duration', 
            }

target_names = {
                'exam_fail' : 'At risk - Exam Grade',
                'final_fail' : 'At risk - Final Grade', 
                'exam_gifted' : 'High performer - Exam Grade', 
                'final_gifted': 'High performer - Final Grade'
                }

#targets
targets = ['exam_fail', 'exam_gifted']

#set the indexes to use for later
index = ["course_encoding", "cd_curso", "semestre", "courseid", "userid", 'exam_gifted', 'exam_fail']

#categories of objecctables
objects = ["course", "resource", "forum", "url", "folder", "quiz", "grade_grades", 
           "assignments", "groups", "user", "turnitintooltwo", "page", "choice", "other"]          

#### Step 3: Import data and take a preliminary look at it 

In [4]:
#imports dataframes
course_programs = pd.read_excel("../Data/Modeling Stage/Nova_IMS_Temporal_Datasets_daily_clicks.xlsx", 
                                dtype = {
                                    'course_encoding' : int,
                                    'userid' : int},
                               sheet_name = 'Date_threshold_100')

#save tables 
student_list = pd.read_csv('../Data/Modeling Stage/Nova_IMS_Filtered_targets.csv', 
                         dtype = {
                                   'course_encoding': int,
                                   'userid' : int,
                                   })

#drop unnamed 0 column
#merge with the targets we calculated on the other 
course_programs = course_programs.merge(student_list, on = ['course_encoding', 'userid'], how = 'inner')
course_programs.drop(['Unnamed: 0', 'exam_mark', 'final_mark'], axis = 1, inplace = True)
    
#convert results to object and need to convert column names to string
course_programs['course_encoding'], course_programs['userid'] = course_programs['course_encoding'].astype(object), course_programs['userid'].astype(object)
course_programs.columns = course_programs.columns.astype(str)

In [5]:
course_programs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63171 entries, 0 to 63170
Columns: 149 entries, course_encoding to exam_gifted
dtypes: float64(95), int64(50), object(4)
memory usage: 72.3+ MB


In [6]:
course_programs.describe(include = 'all')

,course_encoding,cd_curso,semestre,courseid,userid,objecttable,1,2,3,4,...,134,135,136,137,138,139,140,141,exam_fail,exam_gifted
count,63171.0,63171.000000,63171,63171.000000,63171.0,63171,63171.000000,63171.000000,63171.000000,63171.000000,...,41654.000000,39423.000000,37318.000000,34876.000000,32366.000000,27005.000000,19821.000000,1054.000000,63171.000000,63171.000000
unique,138.0,NaN,6,NaN,1590.0,14,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,150.0,NaN,S1,NaN,6826.0,course,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1821.0,NaN,28407,NaN,93.0,9295,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,7906.809375,NaN,185361.922290,NaN,NaN,0.019803,0.028716,0.045638,0.052113,...,0.522903,0.293027,0.235329,0.867387,0.250695,0.844844,0.339690,0.001898,0.190784,0.287331
std,NaN,1986.226115,NaN,80819.428212,NaN,NaN,0.276235,0.308394,0.458564,0.550856,...,4.502103,2.288393,1.885921,9.354308,1.631869,8.256497,3.054003,0.043540,0.392922,0.452521
min,NaN,859.000000,NaN,100001.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,7512.000000,NaN,100091.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,NaN,9155.000000,NaN,200165.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,NaN,9434.000000,NaN,200193.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In our first attempt, we will use the absolute number of clicks made by each student - scaled using standard scaler. 
Therefore, we can start by immediately placing our course encoding/userid pairings into the index.

In [7]:
def normalize(train, test, scaler):
    
    if scaler == 'MinMax':
        pt = MinMaxScaler()
    elif scaler == 'Standard':
        pt = StandardScaler()
    elif scaler == 'Robust':
        pt = RobustScaler()
    elif scaler == 'Quantile':
        pt = QuantileTransformer()
    else:
        pt = PowerTransformer(method='yeo-johnson')
    
    data_train = pt.fit_transform(train)
    data_test = pt.transform(test)
    # convert the array back to a dataframe
    normalized_train = pd.DataFrame(data_train,columns=train.columns)
    normalized_test = pd.DataFrame(data_test,columns=test.columns)
        
    return normalized_train, normalized_test 

In [8]:
course_programs.objecttable.value_counts()

course             9295
resource           8757
forum              7005
url                6769
folder             5758
quiz               4870
grade_grades       4670
assignments        3858
groups             3525
user               2344
turnitintooltwo    1998
page               1728
choice             1373
other              1221
Name: objecttable, dtype: int64

In [9]:
test = course_programs.copy()

#The first 6 columns are index - column 141 is fully empty
columns = test.drop(targets, axis = 1).columns[6:146]

In [10]:
#create first pivot
placeholder_pivot = pd.pivot_table(test, index = index, values = columns, columns = "objecttable",
                  aggfunc = 'first')


#applies the function that removes multiindex
placeholder_pivot.columns = placeholder_pivot.columns.map(flattenHierarchicalCol)

#also saving index for reindexing of the remaining stuff
save_index = placeholder_pivot.index.copy()

#we will need to create the multidimensional tensors
placeholder_dict = {}

#create dataset for targets
df_targets = placeholder_pivot.reset_index().copy()[index]
df_targets.set_index(["course_encoding", "cd_curso", "semestre", "courseid", "userid"], inplace = True)

#initialize empty 3d array
nd_array_100 = np.zeros((
                               len(objects), #nbr of dimensions
                               len(placeholder_pivot), #nbr of rows
                               len(columns), #nbr of columns 
                              ))

#likely inefficient, but should do the trick
counter = 0

#create multiple dataframes based on regex - this will create ndarray for the 100 duration
for i in objects:
    #create the objects
    placeholder_dict[f'{i}'] = placeholder_pivot.filter(regex=f'_{i}')
    
    #remove text, convert column name back to numbers and sort numbers to ensure sequence
    placeholder_dict[f'{i}'].columns = placeholder_dict[f'{i}'].columns.str.replace(r"\D+", "", regex=True) 
    placeholder_dict[f'{i}'].columns = placeholder_dict[f'{i}'].columns.astype(int)
    placeholder_dict[f'{i}'] = placeholder_dict[f'{i}'][sorted(placeholder_dict[f'{i}'].columns)].fillna(0)
    
    #converting df to nd array
    nd_array_100[counter] = placeholder_dict[f'{i}'].values
    counter += 1

    #reshape to samples, rows, columns

#switching to rows, columns, features
nd_array_100 = nd_array_100.transpose(1,2,0)

In [11]:
nd_array_100.shape

(9296, 140, 14)

#### Implementing Cross-Validation with Deep Learning Model

**1. Create the Deep Learning Model**

In this instance, we will follow-up with on the approach used in Chen & Cui - CrossEntropyLoss with applied over a softmax layer.

In [12]:
class LSTM_Uni(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length, dropout):
        super(LSTM_Uni, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first = True) #lstm
        
        self.dropout = nn.Dropout(p = dropout)
    
        self.fc = nn.Linear(self.hidden_size, num_classes) #fully connected last layer

    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)) #internal state
        
        #Xavier_init for both H_0 and C_0
        torch.nn.init.xavier_normal_(h_0)
        torch.nn.init.xavier_normal_(c_0)
        
        # Propagate input through LSTM
        lstm_out, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        last_output = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        
        #we are interested in only keeping the last output
        drop_out = self.dropout(last_output)
        pre_bce = self.fc(drop_out) #Final Output - dense
        return pre_bce

**2. Define the train and validation Functions**

In [13]:
def train_epoch(model,dataloader,loss_fn,optimizer):
    
    train_loss,train_correct=0.0,0 
    model.train()
    for X, labels in dataloader:

        optimizer.zero_grad()
        output = model(X)
        loss = loss_fn(output,labels.unsqueeze(1))
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * X.size(0)
        scores = F.sigmoid(output)
        predictions = torch.round(scores)
        #calculate % correct
        train_correct += (predictions == labels.unsqueeze(1)).sum().item()
        
    return train_loss,train_correct
  
def valid_epoch(model,dataloader,loss_fn):
    valid_loss, val_correct = 0.0, 0
    targets = []
    y_pred = []
    probability_1 = []
    
    model.eval()
    for X, labels in dataloader:

        output = model(X)
        loss=loss_fn(output,labels.unsqueeze(1))
        valid_loss+=loss.item()*X.size(0)
        scores = F.sigmoid(output)
        predictions = torch.round(scores)
        val_correct+=(predictions == labels.unsqueeze(1)).sum().item()
        targets.append(labels.unsqueeze(1))
        y_pred.append(predictions)
        probability_1.append(scores)
        
    #concat all results
    targets = torch.cat(targets).data.cpu().numpy()
    y_pred = torch.cat(y_pred).data.cpu().numpy()
    probability_1 = torch.cat(probability_1).data.cpu().numpy()
    
    #calculate precision, recall and AUC score
    
    precision = precision_score(targets, y_pred)
    recall = recall_score(targets, y_pred)
    f1 = f1_score(targets, y_pred)
    auroc = roc_auc_score(targets, probability_1)
    
    #return all
    return valid_loss,val_correct, precision, recall, auroc, f1

**3. Define main hyperparameters of the model, including splits**

In [14]:
#randomly select from params dict
params = {
    'learning_rate': [0.001, 0.01, 0.005, 0.0005],
    'num_epochs' : [30, 40, 50, 60, 70, 80, 90, 100],
    'hidden_size': [32, 64, 128, 256],
    'batch_size': [32, 64, 128, 256, 512],
    'num_layers': [1, 2, 3],
    'dropout': [0.2, 0.5, 0.7]
    }

from random import choice

combination = [choice(params['learning_rate']), #choose lr
               choice(params['num_epochs']), #max epochs
               choice(params['hidden_size']), #hidden layer size
               choice(params['batch_size']), #hidden batch size
               choice(params['num_layers']), #hidden batch size
               choice(params['dropout'])], # select random_combination of dropout

#Parameters chosen are
combination

([0.005, 50, 64, 128, 1, 0.2],)

In [15]:
#initial configuation - the default
learning_rate = combination[0][0]
num_epochs = combination[0][1]
input_size = 14 #number of features
hidden_size = combination[0][2]
batch_size = combination[0][3]
num_layers = combination[0][4]
dropout = combination[0][5]


#Shape of Output as required for Sigmoid Classifier
num_classes = 1 #output shape

k=10
splits= StratifiedKFold(n_splits=k, random_state=15, shuffle = True) #kfold of 10 with 30 replicas
criterion = nn.BCEWithLogitsLoss()    # cross-entropy for classification

### Test on data - starting with the first 25 days

In [16]:
#create main dict results for 25 days
target_df_dict_25 = {}

for k in tqdm(targets):
    print(k)
    
    #create main dict results
    
    target_df_dict_25[f'{(k)}'] = {}
    y = df_targets[k].values

    #create a list containing one value per row
    all_indices = list(range(len(df_targets)))
    
    #using train test split to later apply the rule accordingly
    train_ind, test_ind = train_test_split(all_indices, test_size=0.2, 
                                           random_state = 5, stratify = y)
    
    #applied train_test_split rules accordingly
    X_train_val = nd_array_100[train_ind,:26,:]
    y_train_val = y[train_ind]
    
    X_test = nd_array_100[test_ind, :26, :]
    y_test = y[test_ind]    
        
    for repeat in range(replicas):
        print('Replica {}'.format(repeat+1))
        #reset "best accuracy for treshold i and target k"
        best_f1_score = 0
        foldperf={}
        
        #make train_val split
        for fold, (train_idx,val_idx) in tqdm(enumerate(splits.split(X_train_val, y_train_val))):
            print('Split {}'.format(fold + 1))
            
            #make split between train and Val
            X_train, y_train = X_train_val[train_idx], y_train_val[train_idx]
            X_val, y_val = X_train_val[val_idx], y_train_val[val_idx]
            
            #scaling requires one scaler per channel (feature)
            scalers = {}
            for feature in range(X_train.shape[2]):
                           
                scalers[feature] = RobustScaler()
                X_train[:, :, feature] = scalers[feature].fit_transform(X_train[:, :, feature]) 

            for col in range(X_val.shape[2]):
                X_val[:, :, feature] = scalers[feature].transform(X_val[:, :, feature]) 
            
            #need to oversample - will use smote
            #will also require oneoversampler per channel (feature)
            samplers = {}
            
            #create new nd_array with the correct size - 2 * majority class 
            X_train_res = np.zeros(shape = (2* (int(sc.stats.mode(y_train)[1])), X_train.shape[1], X_train.shape[2]))
            for feature in range(X_train.shape[2]):
                           
                samplers[feature] = SMOTE()
                X_train_res[:, :, feature], y_train_res = samplers[feature].fit_resample(X_train[:, :, feature], y_train) 
            
            #second, convert everything to pytorch tensor - we will convert to tensor dataset and 
            X_train_tensors = torch.from_numpy(X_train_res)
            X_val_tensors = torch.from_numpy(X_val)
            
            #convert X tensors to format FloatTensor
            X_train_tensors = X_train_tensors.type(torch.cuda.FloatTensor)
            X_val_tensors = X_val_tensors.type(torch.cuda.FloatTensor)
            
            #create y_tensor
            y_train_tensors = torch.from_numpy(y_train_res)
            y_val_tensors = torch.from_numpy(y_val)
            
            #convert y tensors to format longtensor
            y_train_tensors = y_train_tensors.type(torch.cuda.FloatTensor)
            y_val_tensors = y_val_tensors.type(torch.cuda.FloatTensor)
            
            #create Tensor Datasets and dataloaders for both Train and Val
            train_dataset = TensorDataset(X_train_tensors, y_train_tensors)
            val_dataset = TensorDataset(X_val_tensors, y_val_tensors)
            train_loader = DataLoader(train_dataset, batch_size=batch_size)
            val_loader = DataLoader(val_dataset, batch_size=batch_size)
    
            #creates new model for each 
            model = LSTM_Uni(num_classes, input_size, hidden_size, num_layers, X_train_tensors.shape[1], dropout).to('cuda') #our lstm class
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
            scheduler = ReduceLROnPlateau(optimizer, 
                                 'min', 
                                 patience = 15,
                                 cooldown = 20,
                                threshold=0.00001,
                                factor = 0.33,
                                verbose = True)
    
            history = {'train_loss': [], 'val_loss': [],'train_acc':[],'val_acc':[], 'precision': [],
                      'recall' : [], 'auroc': [], 'f1_score' : []}

            for epoch in tqdm(range(num_epochs)):
                train_loss, train_correct=train_epoch(model,train_loader,criterion,optimizer)
                val_loss, val_correct, precision, recall, auroc, f1 = valid_epoch(model,val_loader,criterion)

                train_loss = train_loss / len(train_loader.sampler)
                train_acc = train_correct / len(train_loader.sampler) * 100
                val_loss = val_loss / len(val_loader.sampler)
                val_acc = val_correct / len(val_loader.sampler) * 100
        
        
                if (epoch+1) % 10 == 0: 
                 print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Validation Loss:{:.3f} AVG Training Acc {:.2f} % AVG Validation Acc {:.2f} %".format(epoch + 1,
                                                                                                             num_epochs,
                                                                                                             train_loss,
                                                                                                             val_loss,
                                                                                                             train_acc,
                                                                                                             val_acc))
                history['train_loss'].append(train_loss)
                history['val_loss'].append(val_loss)
                history['train_acc'].append(train_acc)
                history['val_acc'].append(val_acc)
                history['precision'].append(precision)
                history['recall'].append(recall)
                history['auroc'].append(auroc)
                history['f1_score'].append(f1)
                scheduler.step(val_loss)
    
                if f1 > best_f1_score:
            
                #replace best accuracy and save best model
                    print(f'New Best F1_score found: {f1*100:.2f}%\nEpoch: {epoch + 1}\n', 
                         f'Accuracy: {val_acc:.2f}\nAUC: {auroc*100:.2f}')
                    best_f1_score = f1
                    best = deepcopy(model)
                    curr_epoch = epoch + 1
                    
            #store fold performance
            foldperf['fold{}'.format(fold+1)] = history
    
        #create dict to store fold performance
        target_df_dict_25[f'{(k)}']['repeat{}'.format(repeat + 1)] = foldperf

  0%|          | 0/2 [00:00<?, ?it/s]

exam_fail
Replica 1


0it [00:00, ?it/s]

Split 1


  0%|          | 0/50 [00:00<?, ?it/s]

...

Epoch:10/50 AVG Training Loss:0.700 AVG Validation Loss:0.707 AVG Training Acc 42.80 % AVG Validation Acc 33.51 %
Epoch:20/50 AVG Training Loss:0.640 AVG Validation Loss:0.725 AVG Training Acc 68.00 % AVG Validation Acc 50.87 %
Epoch:30/50 AVG Training Loss:0.437 AVG Validation Loss:0.619 AVG Training Acc 79.12 % AVG Validation Acc 69.85 %
Epoch:40/50 AVG Training Loss:0.462 AVG Validation Loss:0.624 AVG Training Acc 79.62 % AVG Validation Acc 67.83 %
Epoch:50/50 AVG Training Loss:0.395 AVG Validation Loss:0.594 AVG Training Acc 81.55 % AVG Validation Acc 70.93 %
Split 9


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch:10/50 AVG Training Loss:0.698 AVG Validation Loss:0.708 AVG Training Acc 43.45 % AVG Validation Acc 32.44 %
Epoch:20/50 AVG Training Loss:0.567 AVG Validation Loss:0.866 AVG Training Acc 70.25 % AVG Validation Acc 48.45 %
Epoch:30/50 AVG Training Loss:0.421 AVG Validation Loss:0.669 AVG Training Acc 79.89 % AVG Validation Acc 68.37 %
Epoch:40/50 AVG Training Loss:0.398 AVG Validation Loss:0.632 AVG Training Acc 81.13 % AVG Validation Acc 70.39 %
Epoch:50/50 AVG Training Loss:0.377 AVG Validation Loss:0.641 AVG Training Acc 82.31 % AVG Validation Acc 71.06 %
Split 10


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch:10/50 AVG Training Loss:0.699 AVG Validation Loss:0.709 AVG Training Acc 43.19 % AVG Validation Acc 27.86 %
Epoch:20/50 AVG Training Loss:0.568 AVG Validation Loss:0.656 AVG Training Acc 74.14 % AVG Validation Acc 67.43 %
Epoch:30/50 AVG Training Loss:0.417 AVG Validation Loss:0.621 AVG Training Acc 80.56 % AVG Validation Acc 70.93 %
Epoch:40/50 AVG Training Loss:0.392 AVG Validation Loss:0.610 AVG Training Acc 81.71 % AVG Validation Acc 70.39 %
Epoch:50/50 AVG Training Loss:0.349 AVG Validation Loss:0.651 AVG Training Acc 84.04 % AVG Validation Acc 66.62 %


In [17]:
#for each target and number of replicas
results_25 = {}

total_results_25 = pd.DataFrame(columns =['Target', 'Repeat', 'Fold', 'Accuracy', 'Precision_(Binary)', 'Recall_(Binary)', 
                                       'F1-score_(Binary)', 'AUC'])

for i in targets:
    results_25[i] = {}
    for k in range(1, replicas + 1):
        results_25[i][f'repeat{k}'] = pd.DataFrame.from_dict(target_df_dict_25[i][f'repeat{k}'], orient = 'index') # convert dict to dataframe
        
        #explode to get eacxh epoch as a row
        results_25[i][f'repeat{k}'] = results_25[i][f'repeat{k}'].explode(list(results_25[i][f'repeat{k}'].columns))
        results_25[i][f'repeat{k}']['fold'] = results_25[i][f'repeat{k}'].index
        results_25[i][f'repeat{k}'].reset_index(drop = True, inplace = True)
        
        #obtain the result that was obtained at the last possible choice
        placeholder_df = results_25[i][f'repeat{k}'].groupby('fold')[['val_acc', 'precision',
                                                                      'recall', 'f1_score', 'auroc']].last().reset_index()
        
        #making adjustments to concat with total_results properly
        placeholder_df.columns = ['Fold', 'Accuracy', 'Precision_(Binary)', 'Recall_(Binary)', 'F1-score_(Binary)', 'AUC']
        placeholder_df = placeholder_df.reindex(columns = total_results_25.columns)
        
        #fill new missing values
        placeholder_df['Target'], placeholder_df['Repeat'] = i, k
        total_results_25 = total_results_25.append(placeholder_df, ignore_index = True)

### Test on data - The first 50 days

Same hyper-parameters.

In [18]:
#create main dict results for 50 days
target_df_dict_50 = {}

for k in tqdm(targets):
    print(k)
    
    #create main dict results
    
    target_df_dict_50[f'{(k)}'] = {}
    y = df_targets[k].values

    #create a list containing one value per row
    all_indices = list(range(len(df_targets)))
    
    #using train test split to later apply the rule accordingly
    train_ind, test_ind = train_test_split(all_indices, test_size=0.2, 
                                           random_state = 5, stratify = y)
    
    #applied train_test_split rules accordingly
    X_train_val = nd_array_100[train_ind,:51,:]
    y_train_val = y[train_ind]
    
    X_test = nd_array_100[test_ind, :51, :]
    y_test = y[test_ind]    
        
    #reset "best accuracy for treshold i and target k"     
    for repeat in range(replicas):
        print('Replica {}'.format(repeat+1))
        
        foldperf={}
        best_f1_score = 0
        
        #make train_val split
        for fold, (train_idx,val_idx) in tqdm(enumerate(splits.split(X_train_val, y_train_val))):
            print('Split {}'.format(fold + 1))
            
            #make split between train and Val
            X_train, y_train = X_train_val[train_idx], y_train_val[train_idx]
            X_val, y_val = X_train_val[val_idx], y_train_val[val_idx]
            
            #scaling requires one scaler per channel (feature)
            scalers = {}
            for feature in range(X_train.shape[2]):
                           
                scalers[feature] = RobustScaler()
                X_train[:, :, feature] = scalers[feature].fit_transform(X_train[:, :, feature]) 

            for col in range(X_val.shape[2]):
                X_val[:, :, feature] = scalers[feature].transform(X_val[:, :, feature]) 
            
            #need to oversample - will use smote
            #will also require oneoversampler per channel (feature)
            samplers = {}
            
            #create new nd_array with the correct size - 2 * majority class 
            X_train_res = np.zeros(shape = (2* (int(sc.stats.mode(y_train)[1])), X_train.shape[1], X_train.shape[2]))
            for feature in range(X_train.shape[2]):
                           
                samplers[feature] = SMOTE()
                X_train_res[:, :, feature], y_train_res = samplers[feature].fit_resample(X_train[:, :, feature], y_train) 
            
            #second, convert everything to pytorch tensor - we will convert to tensor dataset and 
            X_train_tensors = torch.from_numpy(X_train_res)
            X_val_tensors = torch.from_numpy(X_val)
            
            #convert X tensors to format FloatTensor
            X_train_tensors = X_train_tensors.type(torch.cuda.FloatTensor)
            X_val_tensors = X_val_tensors.type(torch.cuda.FloatTensor)
            
            #create y_tensor
            y_train_tensors = torch.from_numpy(y_train_res)
            y_val_tensors = torch.from_numpy(y_val)
            
            #convert y tensors to format longtensor
            y_train_tensors = y_train_tensors.type(torch.cuda.FloatTensor)
            y_val_tensors = y_val_tensors.type(torch.cuda.FloatTensor)
            
            #create Tensor Datasets and dataloaders for both Train and Val
            train_dataset = TensorDataset(X_train_tensors, y_train_tensors)
            val_dataset = TensorDataset(X_val_tensors, y_val_tensors)
            train_loader = DataLoader(train_dataset, batch_size=batch_size)
            val_loader = DataLoader(val_dataset, batch_size=batch_size)
    
            #creates new model for each 
            model = LSTM_Uni(num_classes, input_size, hidden_size, num_layers, X_train_tensors.shape[1], dropout).to('cuda') #our lstm class
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
            scheduler = ReduceLROnPlateau(optimizer, 
                                 'min', 
                                 patience = 15,
                                 cooldown = 20,
                                threshold=0.00001,
                                factor = 0.33,
                                verbose = True)
    
            history = {'train_loss': [], 'val_loss': [],'train_acc':[],'val_acc':[], 'precision': [],
                      'recall' : [], 'auroc': [], 'f1_score' : []}

            for epoch in tqdm(range(num_epochs)):
                train_loss, train_correct=train_epoch(model,train_loader,criterion,optimizer)
                val_loss, val_correct, precision, recall, auroc, f1 = valid_epoch(model,val_loader,criterion)

                train_loss = train_loss / len(train_loader.sampler)
                train_acc = train_correct / len(train_loader.sampler) * 100
                val_loss = val_loss / len(val_loader.sampler)
                val_acc = val_correct / len(val_loader.sampler) * 100
        
        
                if (epoch+1) % 10 == 0: 
                 print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Validation Loss:{:.3f} AVG Training Acc {:.2f} % AVG Validation Acc {:.2f} %".format(epoch + 1,
                                                                                                             num_epochs,
                                                                                                             train_loss,
                                                                                                             val_loss,
                                                                                                             train_acc,
                                                                                                             val_acc))
                history['train_loss'].append(train_loss)
                history['val_loss'].append(val_loss)
                history['train_acc'].append(train_acc)
                history['val_acc'].append(val_acc)
                history['precision'].append(precision)
                history['recall'].append(recall)
                history['auroc'].append(auroc)
                history['f1_score'].append(f1)
                scheduler.step(val_loss)
    
                if f1 > best_f1_score:
            
                #replace best accuracy and save best model
                    print(f'New Best F1_score found: {f1*100:.2f}%\nEpoch: {epoch + 1}\n', 
                         f'Accuracy: {val_acc:.2f}\nAUC: {auroc*100:.2f}')
                    best_f1_score = f1
                    best = deepcopy(model)
                    curr_epoch = epoch + 1
                    
            #store fold performance
            foldperf['fold{}'.format(fold+1)] = history
    
        #create dict to store fold performance
        target_df_dict_50[f'{(k)}']['repeat{}'.format(repeat + 1)] = foldperf

  0%|          | 0/2 [00:00<?, ?it/s]

exam_fail
Replica 1


0it [00:00, ?it/s]

Split 1


  0%|          | 0/50 [00:00<?, ?it/s]

...

Epoch:10/50 AVG Training Loss:0.700 AVG Validation Loss:0.708 AVG Training Acc 43.11 % AVG Validation Acc 37.42 %
Epoch:20/50 AVG Training Loss:0.630 AVG Validation Loss:0.684 AVG Training Acc 67.12 % AVG Validation Acc 57.74 %
Epoch:30/50 AVG Training Loss:0.438 AVG Validation Loss:0.633 AVG Training Acc 79.45 % AVG Validation Acc 69.45 %
Epoch:40/50 AVG Training Loss:0.403 AVG Validation Loss:0.610 AVG Training Acc 80.81 % AVG Validation Acc 70.66 %
Epoch:50/50 AVG Training Loss:0.385 AVG Validation Loss:0.610 AVG Training Acc 81.70 % AVG Validation Acc 70.52 %
Split 9


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch:10/50 AVG Training Loss:0.699 AVG Validation Loss:0.709 AVG Training Acc 45.33 % AVG Validation Acc 41.32 %
Epoch:20/50 AVG Training Loss:0.640 AVG Validation Loss:0.653 AVG Training Acc 67.22 % AVG Validation Acc 64.60 %
Epoch:30/50 AVG Training Loss:0.424 AVG Validation Loss:0.627 AVG Training Acc 80.28 % AVG Validation Acc 70.26 %
Epoch:40/50 AVG Training Loss:0.399 AVG Validation Loss:0.597 AVG Training Acc 81.38 % AVG Validation Acc 72.14 %
Epoch:50/50 AVG Training Loss:0.383 AVG Validation Loss:0.594 AVG Training Acc 81.79 % AVG Validation Acc 72.14 %
Split 10


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch:10/50 AVG Training Loss:0.699 AVG Validation Loss:0.702 AVG Training Acc 45.49 % AVG Validation Acc 46.97 %
Epoch:20/50 AVG Training Loss:0.597 AVG Validation Loss:0.747 AVG Training Acc 69.80 % AVG Validation Acc 56.39 %
Epoch:30/50 AVG Training Loss:0.427 AVG Validation Loss:0.627 AVG Training Acc 79.52 % AVG Validation Acc 71.87 %
Epoch:40/50 AVG Training Loss:0.405 AVG Validation Loss:0.584 AVG Training Acc 80.76 % AVG Validation Acc 72.27 %
Epoch:50/50 AVG Training Loss:0.391 AVG Validation Loss:0.582 AVG Training Acc 81.30 % AVG Validation Acc 72.27 %


In [19]:
#for each target and number of replicas
results_50 = {}

total_results_50 = pd.DataFrame(columns =['Target', 'Repeat', 'Fold', 'Accuracy', 'Precision_(Binary)', 'Recall_(Binary)', 
                                       'F1-score_(Binary)', 'AUC'])

for i in targets:
    results_50[i] = {}
    for k in range(1, replicas + 1):
        results_50[i][f'repeat{k}'] = pd.DataFrame.from_dict(target_df_dict_50[i][f'repeat{k}'], orient = 'index') # convert dict to dataframe
        
        #explode to get eacxh epoch as a row
        results_50[i][f'repeat{k}'] = results_50[i][f'repeat{k}'].explode(list(results_50[i][f'repeat{k}'].columns))
        results_50[i][f'repeat{k}']['fold'] = results_50[i][f'repeat{k}'].index
        results_50[i][f'repeat{k}'].reset_index(drop = True, inplace = True)
        
        #obtain the result that was obtained at the last possible choice
        placeholder_df = results_50[i][f'repeat{k}'].groupby('fold')[['val_acc', 'precision',
                                                                      'recall', 'f1_score', 'auroc']].last().reset_index()
        
        #making adjustments to concat with total_results properly
        placeholder_df.columns = ['Fold', 'Accuracy', 'Precision_(Binary)', 'Recall_(Binary)', 'F1-score_(Binary)', 'AUC']
        placeholder_df = placeholder_df.reindex(columns = total_results_50.columns)
        
        #fill new missing values
        placeholder_df['Target'], placeholder_df['Repeat'] = i, k
        total_results_50 = total_results_50.append(placeholder_df, ignore_index = True)

### Test on data - The first 75 days

Same hyper-parameters.

In [20]:
#create main dict results for 75 days
target_df_dict_75 = {}

for k in tqdm(targets):
    print(k)
    
    #create main dict results
    
    target_df_dict_75[f'{(k)}'] = {}
    y = df_targets[k].values

    #create a list containing one value per row
    all_indices = list(range(len(df_targets)))
    
    #using train test split to later apply the rule accordingly
    train_ind, test_ind = train_test_split(all_indices, test_size=0.2, 
                                           random_state = 5, stratify = y)
    
    #applied train_test_split rules accordingly
    X_train_val = nd_array_100[train_ind,:76,:]
    y_train_val = y[train_ind]
    
    X_test = nd_array_100[test_ind, :76, :]
    y_test = y[test_ind]    
        
    #reset "best accuracy for treshold i and target k"     
    for repeat in range(replicas):
        print('Replica {}'.format(repeat+1))
        
        foldperf={}
        best_f1_score = 0
        
        #make train_val split
        for fold, (train_idx,val_idx) in tqdm(enumerate(splits.split(X_train_val, y_train_val))):
            print('Split {}'.format(fold + 1))
            
            #make split between train and Val
            X_train, y_train = X_train_val[train_idx], y_train_val[train_idx]
            X_val, y_val = X_train_val[val_idx], y_train_val[val_idx]
            
            #scaling requires one scaler per channel (feature)
            scalers = {}
            for feature in range(X_train.shape[2]):
                           
                scalers[feature] = RobustScaler()
                X_train[:, :, feature] = scalers[feature].fit_transform(X_train[:, :, feature]) 

            for col in range(X_val.shape[2]):
                X_val[:, :, feature] = scalers[feature].transform(X_val[:, :, feature]) 
            
            #need to oversample - will use smote
            #will also require oneoversampler per channel (feature)
            samplers = {}
            
            #create new nd_array with the correct size - 2 * majority class 
            X_train_res = np.zeros(shape = (2* (int(sc.stats.mode(y_train)[1])), X_train.shape[1], X_train.shape[2]))
            for feature in range(X_train.shape[2]):
                           
                samplers[feature] = SMOTE()
                X_train_res[:, :, feature], y_train_res = samplers[feature].fit_resample(X_train[:, :, feature], y_train) 
            
            #second, convert everything to pytorch tensor - we will convert to tensor dataset and 
            X_train_tensors = torch.from_numpy(X_train_res)
            X_val_tensors = torch.from_numpy(X_val)
            
            #convert X tensors to format FloatTensor
            X_train_tensors = X_train_tensors.type(torch.cuda.FloatTensor)
            X_val_tensors = X_val_tensors.type(torch.cuda.FloatTensor)
            
            #create y_tensor
            y_train_tensors = torch.from_numpy(y_train_res)
            y_val_tensors = torch.from_numpy(y_val)
            
            #convert y tensors to format longtensor
            y_train_tensors = y_train_tensors.type(torch.cuda.FloatTensor)
            y_val_tensors = y_val_tensors.type(torch.cuda.FloatTensor)
            
            #create Tensor Datasets and dataloaders for both Train and Val
            train_dataset = TensorDataset(X_train_tensors, y_train_tensors)
            val_dataset = TensorDataset(X_val_tensors, y_val_tensors)
            train_loader = DataLoader(train_dataset, batch_size=batch_size)
            val_loader = DataLoader(val_dataset, batch_size=batch_size)
    
            #creates new model for each 
            model = LSTM_Uni(num_classes, input_size, hidden_size, num_layers, X_train_tensors.shape[1], dropout).to('cuda') #our lstm class
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
            scheduler = ReduceLROnPlateau(optimizer, 
                                 'min', 
                                 patience = 15,
                                 cooldown = 20,
                                threshold=0.00001,
                                factor = 0.33,
                                verbose = True)
    
            history = {'train_loss': [], 'val_loss': [],'train_acc':[],'val_acc':[], 'precision': [],
                      'recall' : [], 'auroc': [], 'f1_score' : []}

            for epoch in tqdm(range(num_epochs)):
                train_loss, train_correct=train_epoch(model,train_loader,criterion,optimizer)
                val_loss, val_correct, precision, recall, auroc, f1 = valid_epoch(model,val_loader,criterion)

                train_loss = train_loss / len(train_loader.sampler)
                train_acc = train_correct / len(train_loader.sampler) * 100
                val_loss = val_loss / len(val_loader.sampler)
                val_acc = val_correct / len(val_loader.sampler) * 100
        
        
                if (epoch+1) % 10 == 0: 
                 print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Validation Loss:{:.3f} AVG Training Acc {:.2f} % AVG Validation Acc {:.2f} %".format(epoch + 1,
                                                                                                             num_epochs,
                                                                                                             train_loss,
                                                                                                             val_loss,
                                                                                                             train_acc,
                                                                                                             val_acc))
                history['train_loss'].append(train_loss)
                history['val_loss'].append(val_loss)
                history['train_acc'].append(train_acc)
                history['val_acc'].append(val_acc)
                history['precision'].append(precision)
                history['recall'].append(recall)
                history['auroc'].append(auroc)
                history['f1_score'].append(f1)
                scheduler.step(val_loss)
    
                if f1 > best_f1_score:
            
                #replace best accuracy and save best model
                    print(f'New Best F1_score found: {f1*100:.2f}%\nEpoch: {epoch + 1}\n', 
                         f'Accuracy: {val_acc:.2f}\nAUC: {auroc*100:.2f}')
                    best_f1_score = f1
                    best = deepcopy(model)
                    curr_epoch = epoch + 1
                    
            #store fold performance
            foldperf['fold{}'.format(fold+1)] = history
    
        #create dict to store fold performance
        target_df_dict_75[f'{(k)}']['repeat{}'.format(repeat + 1)] = foldperf

  0%|          | 0/2 [00:00<?, ?it/s]

exam_fail
Replica 1


0it [00:00, ?it/s]

Split 1


  0%|          | 0/50 [00:00<?, ?it/s]

...

Epoch:10/50 AVG Training Loss:0.701 AVG Validation Loss:0.703 AVG Training Acc 40.12 % AVG Validation Acc 29.07 %
Epoch:20/50 AVG Training Loss:0.611 AVG Validation Loss:0.714 AVG Training Acc 70.83 % AVG Validation Acc 62.18 %
Epoch    25: reducing learning rate of group 0 to 1.6500e-03.
Epoch:30/50 AVG Training Loss:0.442 AVG Validation Loss:0.629 AVG Training Acc 80.15 % AVG Validation Acc 69.58 %
Epoch:40/50 AVG Training Loss:0.411 AVG Validation Loss:0.610 AVG Training Acc 80.99 % AVG Validation Acc 70.39 %
Epoch:50/50 AVG Training Loss:0.396 AVG Validation Loss:0.605 AVG Training Acc 81.57 % AVG Validation Acc 70.79 %
Split 9


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch:10/50 AVG Training Loss:0.702 AVG Validation Loss:0.703 AVG Training Acc 39.05 % AVG Validation Acc 27.86 %
Epoch:20/50 AVG Training Loss:0.626 AVG Validation Loss:0.809 AVG Training Acc 68.48 % AVG Validation Acc 46.16 %
Epoch:30/50 AVG Training Loss:0.408 AVG Validation Loss:0.596 AVG Training Acc 81.02 % AVG Validation Acc 71.87 %
Epoch:40/50 AVG Training Loss:0.394 AVG Validation Loss:0.595 AVG Training Acc 81.69 % AVG Validation Acc 72.41 %
Epoch:50/50 AVG Training Loss:0.381 AVG Validation Loss:0.601 AVG Training Acc 82.38 % AVG Validation Acc 71.87 %
Split 10


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch:10/50 AVG Training Loss:0.699 AVG Validation Loss:0.702 AVG Training Acc 40.38 % AVG Validation Acc 28.26 %
New Best F1_score found: 44.53%
Epoch: 14
 Accuracy: 39.30
AUC: 58.65
Epoch:20/50 AVG Training Loss:0.615 AVG Validation Loss:0.613 AVG Training Acc 71.58 % AVG Validation Acc 69.99 %
Epoch:30/50 AVG Training Loss:0.411 AVG Validation Loss:0.605 AVG Training Acc 81.42 % AVG Validation Acc 71.60 %
Epoch:40/50 AVG Training Loss:0.409 AVG Validation Loss:0.582 AVG Training Acc 81.08 % AVG Validation Acc 73.49 %
Epoch:50/50 AVG Training Loss:0.393 AVG Validation Loss:0.580 AVG Training Acc 81.81 % AVG Validation Acc 73.35 %


In [21]:
#for each target and number of replicas
results_75 = {}

total_results_75 = pd.DataFrame(columns =['Target', 'Repeat', 'Fold', 'Accuracy', 'Precision_(Binary)', 'Recall_(Binary)', 
                                       'F1-score_(Binary)', 'AUC'])

for i in targets:
    results_75[i] = {}
    for k in range(1, replicas + 1):
        results_75[i][f'repeat{k}'] = pd.DataFrame.from_dict(target_df_dict_75[i][f'repeat{k}'], orient = 'index') # convert dict to dataframe
        
        #explode to get eacxh epoch as a row
        results_75[i][f'repeat{k}'] = results_75[i][f'repeat{k}'].explode(list(results_75[i][f'repeat{k}'].columns))
        results_75[i][f'repeat{k}']['fold'] = results_75[i][f'repeat{k}'].index
        results_75[i][f'repeat{k}'].reset_index(drop = True, inplace = True)
        
        #obtain the result that was obtained at the last possible choice
        placeholder_df = results_75[i][f'repeat{k}'].groupby('fold')[['val_acc', 'precision',
                                                                      'recall', 'f1_score', 'auroc']].last().reset_index()
        
        #making adjustments to concat with total_results properly
        placeholder_df.columns = ['Fold', 'Accuracy', 'Precision_(Binary)', 'Recall_(Binary)', 'F1-score_(Binary)', 'AUC']
        placeholder_df = placeholder_df.reindex(columns = total_results_75.columns)
        
        #fill new missing values
        placeholder_df['Target'], placeholder_df['Repeat'] = i, k
        total_results_75 = total_results_75.append(placeholder_df, ignore_index = True)

### Saving results

In [24]:
#saving agregate results in Excel File
with pd.ExcelWriter('../Data/Modeling Stage/Results/IMS/Clicks per day/model_2_results.xlsx') as writer:
    total_results_25.to_excel(writer, sheet_name = 'total_results_25', index = False)
    total_results_50.to_excel(writer, sheet_name = 'total_results_50', index = False)
    total_results_75.to_excel(writer, sheet_name = 'total_results_75', index = False)
    
#saving raw_results in json file
import json
daily_clicks_raw_results = [target_df_dict_25, 
               target_df_dict_50, 
               target_df_dict_75]

# e.g. file = './data.json' 
with open("../Data/Modeling Stage/Results/IMS/Clicks per day/model_2_raw_results.json", 'w') as f: 
    json.dump(daily_clicks_raw_results, f,  indent=4)